# Thermal rates calculation and comparison

This notebook calculates and compares the thermal rates using data from C. Vogl's version of TARDIS. The data were acquired by running his code using PDB and extracting dataframes as CSV. Atomic data used is listed in a following cell.

In [ ]:
from pathlib import Path

import pandas as pd

home = Path.home()

# input data from C Vogl's version of TARDIS
electron_temp = [9992.2722969523056, 9992.592241054022, ]
electron_density = [2.20676447e09, 1624998247.515976, ]

ff_heating_estimator = [  4.89135279e-24,   4.37696370e-24,   3.75869301e-24,
         4.97847160e-24,   4.52158002e-24,   4.21024499e-24,
         3.94991540e-24,   3.72915649e-24,   3.58902110e-24,
         3.40170224e-24,   3.20848519e-24,   3.03540032e-24,
         2.87314722e-24,   2.74328938e-24,   2.61063140e-24,
         2.50640248e-24,   2.38164559e-24,   2.26967531e-24,
         2.24509826e-24,   2.12378192e-24,   2.02063266e-24,
         1.92509873e-24,   1.83070678e-24,   1.77346374e-24]
data_path = home / "tardis-regression-data/testdata/thermal_data"
bf_heating_estimator = pd.read_csv(data_path / "thermal_bf_heating_est.csv", index_col=(0, 1, 2))
stim_recomb_cooling_estimator = pd.read_csv(data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2))
ion_number_density = pd.read_csv(data_path / "thermal_ion_number_density.csv", index_col=(0, 1))
level_number_density = pd.read_csv(data_path / "thermal_level_number_density.csv", index_col=(0, 1, 2))
level_population_ratio = pd.read_csv(data_path / "thermal_level_pop_ratio.csv", index_col=(0, 1, 2))
coll_exc_coeff = pd.read_csv(data_path / "thermal_coll_exc_coeff.csv", index_col=(0, 1, 2, 3))
coll_deexc_coeff = pd.read_csv(data_path / "thermal_coll_deexc_coeff.csv", index_col=(0, 1, 2, 3))
lines = pd.read_csv(data_path / "thermal_lines.csv", index_col=(0))
coll_ion_rate_coeff = pd.read_csv(data_path / "thermal_coll_ion_rate_coeff.csv", index_col=(0, 1, 2))

# because pandas reads in the columns as strings, we need to convert them back to integers
bf_heating_estimator.columns = bf_heating_estimator.columns.astype(int)
stim_recomb_cooling_estimator.columns = stim_recomb_cooling_estimator.columns.astype(int)
ion_number_density.columns = ion_number_density.columns.astype(int)
level_number_density.columns = level_number_density.columns.astype(int)
level_population_ratio.columns = level_population_ratio.columns.astype(int)
coll_exc_coeff.columns = coll_exc_coeff.columns.astype(int)
coll_deexc_coeff.columns = coll_deexc_coeff.columns.astype(int)
lines.index = lines.index.astype(int)
coll_ion_rate_coeff.columns = coll_ion_rate_coeff.columns.astype(int)

Benchmark values from CTARDIS acquired using PDB and looping through shells. Only 2 shells tested against due to having to loop through shells and manually copy data.

In [ ]:
fb_cooling_ct = [1.2018593543520837e-06, 6.467703748685086e-07, ]
ff_cooling_ct = [6.941664530316456e-07, 3.7641281653486813e-07, ]
coll_ion_cooling_ct = [1.6125333965984259e-07, 6.45338605094838e-08, ]
bf_heating_ct = [1.2809489753862688e-06, 7.017461756945128e-07, ]
ff_heating_ct = [2.3829164962085199e-07, 1.1562178134581216e-07, ]
coll_ion_heating_ct = [1.5946196993219911e-07, 6.366986972173936e-08, ]
total_heating_rate_ct = [-3.8611749800759465e-07, -2.100641079650486e-07, ]
fractional_heating_rate_ct = [-0.036553097452112299, -0.04676864172507447, ]

coll_exc_cooling_rate_ct = [8.5059159013e-06, 3.403841826360594e-06, ]
coll_deexc_heating_rate_ct = [8.49837495539e-06, 3.400456943546342e-06, ]

In [ ]:
import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.plasma.electron_energy_distribution import (
         ThermalElectronEnergyDistribution,
)
from tardis.plasma.equilibrium.rates.heating_cooling_rates import (
         BoundFreeThermalRates,
         CollisionalBoundThermalRates,
         CollisionalIonizationThermalRates,
         FreeFreeThermalRates,
)

# identical atomic data to that used by C Vogl
atom_data = AtomData.from_hdf(home / "tardis-regression-data/atom_data/nlte_atom_data/TestNLTE_He_Ti_ctardis.h5") # currently not available for public use
atom_data.prepare_atom_data([1], "macroatom", [], [(1, 0)])

# matching electron distribution to that used by C Vogl
thermal_electron_distribution = ThermalElectronEnergyDistribution(0 * u.erg, electron_temp * u.K, electron_density * u.cm**-3)

/home/afullard/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

## Solver comparison for each process

Each solver is set up using the appropriate inputs and compared to the C Vogl output. All inputs are from C Vogl code.

In [4]:
bf_rates = BoundFreeThermalRates(atom_data.photoionization_data)
bf_heating, bf_cooling = bf_rates.solve(
    level_number_density.loc[:, :1],
    ion_number_density.loc[:, :1],
    thermal_electron_distribution,
    level_population_ratio.loc[:, :1],
    bound_free_heating_estimator=bf_heating_estimator.loc[:, :1],
    stimulated_recombination_estimator=stim_recomb_cooling_estimator.loc[:, :1],
)

In [5]:
(bf_heating - bf_heating_ct) / bf_heating_ct

0   -4.034643e-12
1   -1.508795e-16
dtype: float64

In [6]:
(bf_cooling - fb_cooling_ct) / fb_cooling_ct

0    8.796714e-07
1    8.863891e-07
dtype: float64

In [7]:
ff_rates = FreeFreeThermalRates()
ff_heating, ff_cooling = ff_rates.solve(
    ff_heating_estimator[:2],
    thermal_electron_distribution,
    ion_number_density.loc[:, :1],
)

In [8]:
(ff_cooling - ff_cooling_ct) / ff_cooling_ct

0   -2.807564e-11
1    0.000000e+00
dtype: float64

In [9]:
(ff_heating - ff_heating_ct) / ff_heating_ct

0   -9.876061e-10
1   -7.579947e-10
dtype: float64

In [10]:
coll_ion_rates = CollisionalIonizationThermalRates(atom_data.photoionization_data)
coll_ion_heating, coll_ion_cooling = coll_ion_rates.solve(
    thermal_electron_distribution.number_density,
    ion_number_density.loc[:, :1],
    level_number_density.loc[:, :1],
    coll_ion_rate_coeff.loc[:, :1],
    level_population_ratio.loc[:, :1],
    )

In [11]:
(coll_ion_heating - coll_ion_heating_ct) / coll_ion_heating_ct

0   -7.098834e-08
1   -7.093212e-08
dtype: float64

In [12]:
(coll_ion_cooling - coll_ion_cooling_ct) / coll_ion_cooling_ct

0   -7.095974e-08
1   -7.093211e-08
dtype: float64

In [ ]:
coll_bound_rates = CollisionalBoundThermalRates(lines)
coll_deexc_heating, coll_exc_cooling = coll_bound_rates.solve(
    thermal_electron_distribution.number_density,
    coll_deexc_coeff.loc[:, :1],
    coll_exc_coeff.loc[:, :1],
    level_number_density.loc[:, :1],
)

In [ ]:
(coll_deexc_heating - coll_deexc_heating_rate_ct) / coll_deexc_heating_rate_ct

0   -7.095960e-08
1   -7.093195e-08
dtype: float64

In [ ]:
(coll_exc_cooling - coll_exc_cooling_rate_ct) / coll_exc_cooling_rate_ct

0   -7.096040e-08
1   -7.093195e-08
dtype: float64

## Thermal balance solution

In [16]:
from tardis.plasma.equilibrium.thermal_balance import ThermalBalanceSolver

thermal_solver = ThermalBalanceSolver(bf_rates, ff_rates, coll_ion_rates, coll_bound_rates)

heating_rate, fractional_heating_rate = thermal_solver.solve(
    thermal_electron_distribution,
    level_number_density.loc[:, :1],
    ion_number_density.loc[:, :1],
    coll_ion_rate_coeff.loc[:, :1],
    coll_deexc_coeff.loc[:, :1],
    coll_exc_coeff.loc[:, :1],
    ff_heating_estimator[:2],
    level_population_ratio.loc[:, :1],
    bound_free_heating_estimator=bf_heating_estimator.loc[:, :1],
    stimulated_recombination_estimator=stim_recomb_cooling_estimator.loc[:, :1],
)

In [17]:
(heating_rate - total_heating_rate_ct) / total_heating_rate_ct

0    0.000003
1    0.000003
dtype: float64

In [18]:
(fractional_heating_rate - fractional_heating_rate_ct) / fractional_heating_rate_ct

0    0.000003
1    0.000003
dtype: float64